In [1]:
import cv2
import numpy as np
import tensorflow as tf

2026-01-29 12:03:52.426232: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def predict_frame(model, frame):
    # resize to model input
    img = cv2.resize(frame, (64, 64))
    
    # convert BGR → RGB (VERY IMPORTANT)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # normalize
    img = img / 255.0
    
    # add batch dimension
    img = np.expand_dims(img, axis=0)
    
    # predict
    prediction = model.predict(img, verbose=0)
    
    return prediction[0][0]


In [5]:
import cv2
from tensorflow import keras
import time

# Load model
model = keras.models.load_model("models/cnn_model.keras")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    prob = predict_frame(model, frame)

    label = "Dog" if prob > 0.5 else "Cat"
    confidence = prob if prob > 0.5 else 1 - prob

    # --- Display text (clean formatting) ---
    cv2.putText(
        frame,
        f"Prediction: {label}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    cv2.putText(
        frame,
        f"Confidence: {confidence:.2f}",
        (20, 80),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,
        (0, 255, 0),
        2
    )

    cv2.imshow("Webcam Prediction", frame)

    key = cv2.waitKey(1) & 0xFF

    # --- Capture frame when user presses 'c' ---
    if key == ord('c'):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        filename = f"capture_{timestamp}.jpg"
        cv2.imwrite(filename, frame)
        print(f"📸 Saved {filename}")

    # --- Quit on 'q' ---
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


📸 Saved capture_20260129_121503.jpg
📸 Saved capture_20260129_121505.jpg
📸 Saved capture_20260129_121540.jpg
📸 Saved capture_20260129_121541.jpg
📸 Saved capture_20260129_121544.jpg
